# Taichi Lang Cheat Sheet

### ⚡️ Handy reminders
- 🔧 Initialise the runtime once, pick a backend, and enable profilers up front.
- 🧊 Fields live on Taichi’s runtime—treat them as your GPU/CPU-resident arrays.
- 🧩 Kernels stay flexible by taking fields as `ti.template()` parameters so you can reuse them across layouts.
- 📈 Profiling and atomics are built-in; you can inspect hotspots and resolve data races without leaving Python.

Run the cells top to bottom; copy the snippets into your own projects as needed.

## Installation and import


In [ ]:
!pip install taichi

In [ ]:
import taichi as ti
import numpy as np

%matplotlib widget


### 🔌 Initialise once
Pick the preferred backend and enable the kernel profiler so you can inspect hotspots after a run.

In [ ]:
ti.init(arch=ti.gpu, kernel_profiler=True, default_fp=ti.f32)

### 🧱 Fields and data types
Define scalar/vector fields, integer masks, and particle buffers directly on the Taichi runtime. They are strongly typed, so decide precision up front.

In [ ]:
grid_shape = (64, 64)
nx, ny = grid_shape
num_particles = 4096

scalar_grid = ti.field(dtype=ti.f32, shape=grid_shape)
scratch_grid = ti.field(dtype=ti.f32, shape=grid_shape)
vector_grid = ti.Vector.field(2, dtype=ti.f32, shape=grid_shape)
mask_grid = ti.field(dtype=ti.i32, shape=grid_shape)
particle_positions = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
particle_weights = ti.field(dtype=ti.f32, shape=num_particles)

print('scalar dtype:', scalar_grid.dtype)
print('vector dtype:', vector_grid.dtype, 'components:', vector_grid.n)
print('particle count:', num_particles)


### ♻️ Template helper funcs
Device helpers can also accept `ti.template()` arguments, which lets them operate on any compatible field or scalar without recompiling.

In [ ]:
@ti.func
def clamp_vec(v: ti.template(), lower: ti.template(), upper: ti.template()):
    return ti.min(ti.max(v, lower), upper)

@ti.func
def laplacian(field: ti.template(), coord: ti.template()):
    i = coord[0]
    j = coord[1]
    return (
        -4.0 * field[i, j]
        + field[i + 1, j]
        + field[i - 1, j]
        + field[i, j + 1]
        + field[i, j - 1]
    )

@ti.func
def gradient(field: ti.template(), coord: ti.template(), spacing: ti.template()):
    i = coord[0]
    j = coord[1]
    gx = (field[i + 1, j] - field[i - 1, j]) * 0.5 / spacing
    gy = (field[i, j + 1] - field[i, j - 1]) * 0.5 / spacing
    return ti.Vector([gx, gy])


### 🧮 Reusable kernels
Use `ti.template()` parameters so the same kernel can work with different fields or constants without rewriting anything.

In [ ]:
@ti.kernel
def fill(field: ti.template(), value: ti.template()):
    for I in ti.grouped(field):
        field[I] = value

@ti.kernel
def add_ramp(field: ti.template(), axis: ti.template(), scale: ti.template()):
    for I in ti.grouped(field):
        field[I] += scale * I[axis]

@ti.kernel
def copy_field(src: ti.template(), dst: ti.template()):
    for I in ti.grouped(src):
        dst[I] = src[I]

@ti.kernel
def write_mask_from_threshold(field: ti.template(), mask: ti.template(), cut: ti.template()):
    for I in ti.grouped(field):
        mask[I] = int(field[I] > cut)


#### ✅ Template kernels in action
Fill a grid, add a ramp along one axis, copy it elsewhere, and flag cells above a threshold.

In [ ]:
fill(scalar_grid, 0.0)
add_ramp(scalar_grid, 1, 0.01)
copy_field(scalar_grid, scratch_grid)
write_mask_from_threshold(scalar_grid, mask_grid, 0.4)

sample_row = 20
print('ramp sample (row 20):', np.round(scalar_grid.to_numpy()[sample_row, :8], 3))
print('mask hits in row 20:', int(mask_grid.to_numpy()[sample_row].sum()))


### 🌀 Complex kernel example
Interior-only iterators plus helper funcs let you build more advanced stencils while keeping the signature reusable.

In [ ]:
@ti.kernel
def diffuse_step(src: ti.template(), dst: ti.template(), alpha: ti.template(), spacing: ti.template()):
    nx_local = ti.static(src.shape[0])
    ny_local = ti.static(src.shape[1])
    for i, j in ti.ndrange((1, nx_local - 1), (1, ny_local - 1)):
        lap = laplacian(src, ti.Vector([i, j]))
        dst[i, j] = src[i, j] + alpha * lap * spacing

@ti.kernel
def advect_gradient(src: ti.template(), dst: ti.template(), spacing: ti.template(), strength: ti.template()):
    nx_local = ti.static(src.shape[0])
    ny_local = ti.static(src.shape[1])
    for i, j in ti.ndrange((1, nx_local - 1), (1, ny_local - 1)):
        grad = gradient(src, ti.Vector([i, j]), spacing)
        dst[i, j] = grad * strength


#### 🎯 Complex kernel demo
Diffuse a ramped field a few times, compute its gradient, and peek at a tiny neighbourhood.

In [ ]:
fill(scalar_grid, 0.0)
add_ramp(scalar_grid, 0, 0.02)
copy_field(scalar_grid, scratch_grid)

for _ in range(8):
    diffuse_step(scratch_grid, scalar_grid, 0.15, 1.0)
    copy_field(scalar_grid, scratch_grid)

advect_gradient(scalar_grid, vector_grid, 1.0, 0.6)
local_patch = scalar_grid.to_numpy()[28:32, 28:32]
local_grad = vector_grid.to_numpy()[28:32, 28:32]
print('scalar patch:', np.round(local_patch, 4))
print('gradient norms:', np.round(np.linalg.norm(local_grad, axis=-1), 4))


### 🔩 Atomic operations
Race-free updates come from `ti.atomic_add`. Here we scatter particle weights into a grid histogram.

In [ ]:
@ti.kernel
def seed_particles(pos: ti.template(), weights: ti.template(), spread: ti.template()):
    for i in pos:
        xi = ti.random(ti.f32) * (nx - 1)
        yi = ti.random(ti.f32) * (ny - 1)
        pos[i] = ti.Vector([xi, yi])
        weights[i] = 1.0 + ti.random(ti.f32) * spread

@ti.kernel
def jitter_particles(pos: ti.template(), strength: ti.template()):
    upper = ti.Vector([nx - 1.001, ny - 1.001])
    for i in pos:
        offset = ti.Vector([ti.random(ti.f32) - 0.5, ti.random(ti.f32) - 0.5]) * strength
        pos[i] = clamp_vec(pos[i] + offset, ti.Vector([0.0, 0.0]), upper)

@ti.kernel
def scatter_to_grid(pos: ti.template(), weights: ti.template(), grid: ti.template()):
    for i in pos:
        cell = ti.cast(pos[i], ti.i32)
        cell = clamp_vec(cell, ti.Vector([0, 0]), ti.Vector([nx - 1, ny - 1]))
        ti.atomic_add(grid[cell[0], cell[1]], weights[i])


#### 🛠️ Atomic scatter demo
Seed particles, jitter them a little, and accumulate their weights into a histogram grid using `ti.atomic_add`.

In [ ]:
fill(scratch_grid, 0.0)
seed_particles(particle_positions, particle_weights, 1.0)
for step in range(3):
    jitter_particles(particle_positions, 1.0)
    scatter_to_grid(particle_positions, particle_weights, scratch_grid)

hist = scratch_grid.to_numpy()
print('histogram min/max:', float(hist.min()), float(hist.max()))
print('non-zero bins:', int((hist > 0).sum()))


### 📊 Profiling walkthrough
Clear the profiler, run a quick pipeline, then print the timing table to spot hotspots.

In [ ]:
ti.profiler.clear_kernel_profiler_info()

fill(scalar_grid, 0.0)
add_ramp(scalar_grid, 0, 0.01)
copy_field(scalar_grid, scratch_grid)

for _ in range(12):
    diffuse_step(scratch_grid, scalar_grid, 0.18, 1.0)
    copy_field(scalar_grid, scratch_grid)

advect_gradient(scalar_grid, vector_grid, 1.0, 0.4)

fill(mask_grid, 0)
write_mask_from_threshold(scalar_grid, mask_grid, 0.05)

fill(scratch_grid, 0.0)
seed_particles(particle_positions, particle_weights, 1.5)
for _ in range(4):
    jitter_particles(particle_positions, 1.2)
    scatter_to_grid(particle_positions, particle_weights, scratch_grid)

np_scalar = scalar_grid.to_numpy()
np_mask = mask_grid.to_numpy()
np_mag = np.linalg.norm(vector_grid.to_numpy(), axis=-1)
print('scalar range:', float(np_scalar.min()), float(np_scalar.max()))
print('mask coverage:', np_mask.sum())
print('gradient max magnitude:', float(np_mag.max()))

print('Profiler results:')
ti.profiler.print_kernel_profiler_info()
